In [1]:
import requests
import pandas as pd
import geopandas as gpd

In [10]:
# define the endpoint
endpoint = 'https://api.yelp.com/v3/businesses/search'
api_key = 'ZsHZFGtKEZeOOXhTne98eErmfd6BfNTm9GqO2S6inSnWzDwtgC2sEauXcB-8zUna_lXZEal4jsW_St6O0OQOcuNvifrr6uqNYmjFXW-FyVKvaMyczbihWELI80tjYHYx' # as a str 
headers = {'Authorization': 'bearer %s' % api_key}
parameters = {
    'term':"supermarket",
    'limit':50,
    'radius':13268,
    'offset':150,
    'latitude':86.64481006479548,
    'longitude': -106.35111223691125
}

response = requests.get(url=endpoint, params=parameters, headers=headers)
result = response.json()

In [11]:
result

{'businesses': [],
 'total': 0,
 'region': {'center': {'longitude': -106.35111223691125,
   'latitude': 86.64481006479548}}}

In [14]:
def get_results(keyword, lat=None, lon=None, location=None, radius=40000):
    assert lat != None and lon != None or location != None
    if location != None:
        parameters = {
        'term': keyword,
        'limit':50,
        'radius':radius,
        'location':'Madison, Wisconsin'}
    else:
        parameters = {
        'term': keyword,
        'limit':50,
        'radius':radius,
        'latitude':lat,
        'longitude':lon}
    all_results = []
    response = requests.get(url=endpoint, params=parameters, headers=headers)
    result = response.json()
    all_results += result['businesses']
    
    found = result['total']
    searches = found // 50 
    
    if searches != 0:
        for i in range(searches):
            offset = (i+1) * 50
            parameters['offset'] = offset
            response = requests.get(url=endpoint, params=parameters, headers=headers)
            result = response.json()
            all_results += result['businesses']
    return all_results


def get_info(data, key='name'):
    final = []
    for item in data:
        obj = item[key]
        final.append(obj)
    return final


def get_dataframe(data):
    df = pd.DataFrame()
    for col in keys:
        df[col] = get_info(data, key=col)
    return df
    

keys = ['id', 'alias', 'name', 'image_url', 'is_closed', 
        'url', 'review_count', 'categories', 'rating', 'coordinates', 
        'transactions', 'location', 'phone', 'display_phone', 'distance']

In [16]:
restaurants = get_results('restaurants', lat=43.073051, lon=-89.401230)

In [17]:
# can input id, alias, coordinates, location (address basically), review count etc
get_info(restaurants, 'name')[:15]

['Graze',
 'Pig in a Fur Coat',
 'Heritage Tavern',
 'Everyday Kitchen - Madison',
 'The Old Fashioned',
 'Settle Down Tavern',
 "Dotty Dumpling's Dowry",
 'Sweet Home Wisconsin',
 'Merchant',
 'Mediterranean Cafe',
 'Nook',
 'The Weary Traveler Freehouse',
 "Paul's Pel'meni",
 'Mr.Kimchi Modern Korean',
 'La Kitchenette']

In [18]:
df = get_dataframe(restaurants)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance
0,XmsBtCykUtObXg4Q1mYpvg,graze-madison,Graze,https://s3-media3.fl.yelpcdn.com/bphoto/2zSpLS...,False,https://www.yelp.com/biz/graze-madison?adjust_...,1174,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 43.0750940450292, 'longitude': -8...",[delivery],"{'address1': '1 S Pinckney St', 'address2': ''...",+16082512700,(608) 251-2700,1571.032819
1,OoI3WUe4Xl86d53avy3oVA,pig-in-a-fur-coat-madison,Pig in a Fur Coat,https://s3-media1.fl.yelpcdn.com/bphoto/3-xM2o...,False,https://www.yelp.com/biz/pig-in-a-fur-coat-mad...,441,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 43.08056, 'longitude': -89.36869}",[delivery],"{'address1': '940 Williamson St', 'address2': ...",+16083163300,(608) 316-3300,2778.536281
2,sXYkjcuGhtaYP8yeQPaHgA,heritage-tavern-madison,Heritage Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/Rn3rWz...,False,https://www.yelp.com/biz/heritage-tavern-madis...,505,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 43.076779, 'longitude': -89.383321}","[restaurant_reservation, delivery, pickup]","{'address1': '131 E Mifflin St', 'address2': '...",+16082839500,(608) 283-9500,1520.081956
3,gXSmo3owUfQXIFduFy7kBQ,everyday-kitchen-madison-madison,Everyday Kitchen - Madison,https://s3-media2.fl.yelpcdn.com/bphoto/U7VjHC...,False,https://www.yelp.com/biz/everyday-kitchen-madi...,49,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 43.07502, 'longitude': -89.43708}","[delivery, pickup]","{'address1': '2801 Marshall Ct', 'address2': N...",+16084705500,(608) 470-5500,2914.811812
4,RJNAeNA-209sctUO0dmwuA,the-old-fashioned-madison,The Old Fashioned,https://s3-media2.fl.yelpcdn.com/bphoto/welglu...,False,https://www.yelp.com/biz/the-old-fashioned-mad...,1916,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.0761862475496, 'longitude': -8...","[delivery, pickup]","{'address1': '23 N Pinckney St', 'address2': '...",+16083104545,(608) 310-4545,1461.399409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,MwsLN3Lo2SEbldy0HvBYEw,subway-madison-97,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/e4lXe9...,False,https://www.yelp.com/biz/subway-madison-97?adj...,3,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",2.5,"{'latitude': 43.1476537619461, 'longitude': -8...",[],"{'address1': '5551 E Park Blvd', 'address2': N...",+16084420113,(608) 442-0113,12362.120519
933,khHxKm1_x6syS5W2LTX0lw,quiznos-madison-12,Quiznos,,False,https://www.yelp.com/biz/quiznos-madison-12?ad...,3,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 43.0697212730165, 'longitude': -8...",[delivery],"{'address1': '116 Gammon Rd', 'address2': '', ...",+16088333434,(608) 833-3434,8233.300716
934,k7vX7EO6a88gMxTAz5tiCw,mcdonalds-mcfarland,McDonald's,https://s3-media2.fl.yelpcdn.com/bphoto/d0i4Pu...,False,https://www.yelp.com/biz/mcdonalds-mcfarland?a...,12,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.0,"{'latitude': 43.017724, 'longitude': -89.299713}",[delivery],"{'address1': '4800 Larson Beach Rd', 'address2...",+16088380139,(608) 838-0139,10290.912769
935,hc81y_5FQjTiZcgqnY8vfQ,pizza-hut-madison-8,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/EqIpdg...,False,https://www.yelp.com/biz/pizza-hut-madison-8?a...,16,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",1.5,"{'latitude': 43.0846214, 'longitude': -89.2997...",[delivery],"{'address1': '4762 Cottage Grove Rd', 'address...",+16082210666,(608) 221-0666,8354.865379
